# Seznam - Predicting the transaction volume

*NOTE: Due to the size of the dataset, this notebook will not run on MyBinder.*

Seznam is a Czech company with a scope similar to Google. The purpose of this notebook is to analyze data from Seznam's wallet, predicting the transaction volume.

Summary:

- Prediction type: __Regression model__
- Domain: __E-commerce__
- Prediction target: __Transaction volume__ 
- Population size: __1,458,233__

_Author: Dr. Patrick Urbanke_

# Background

Seznam is a Czech company with a scope similar to Google. The purpose of this notebook is to analyze data from Seznam's wallet, predicting the transaction volume.

Since the dataset is in Czech, we will quickly translate the meaning of the main tables:

- *dobito*: contains data on prepayments into a wallet
- *probehnuto*: contains data on charges from a wallet
- *probehnuto_mimo_penezenku*: contains data on charges, from sources other than a wallet

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/Seznam) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import featuretools
import getml

getml.engine.launch()
getml.set_project('seznam')

Launched the getML engine. The log output will be stored in /home/patrick/.getML/logs/20220322202946.log.


Loading pipelines...
[========================================] 100%


Connected to project 'seznam'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="Seznam",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='Seznam',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
dobito = load_if_needed("dobito")
probehnuto = load_if_needed("probehnuto")
probehnuto_mimo_penezenku = load_if_needed("probehnuto_mimo_penezenku")

In [5]:
dobito

name      client_id   month_year_datum_transakce   sluzba          kc_dobito    
  role   unused_float   unused_string                unused_string   unused_string
     0        7157857   2012-10-01                   c               1045.62      
     1         109700   2015-10-01                   c               5187.28      
     2          51508   2015-08-01                   c               408.20       
     3        9573550   2012-10-01                   c               521.24       
     4        9774621   2014-11-01                   c               386.22       
                  ...   ...                          ...             ...          
554341          65283   2012-09-01                   g               7850.00      
554342        6091446   2012-08-01                   g               31400.00     
554343        1264806   2013-08-01                   g               -8220.52     
554344         101103   2012-08-01                   g               3140.00      
554345        8674551   2012-08-01                   g               6280.00      


554346 rows x 4 columns
memory usage: 29.59 MB
name: dobito
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/seznam/dobito/

In [6]:
probehnuto

name      client_id   month_year_datum_transakce   sluzba          kc_proklikano
   role   unused_float   unused_string                unused_string   unused_string
      0         109145   2013-06-01                   c               -31.40       
      1        9804394   2015-10-01                   h               37.68        
      2        9803353   2015-10-01                   h               725.34       
      3        9801753   2015-10-01                   h               194.68       
      4        9800425   2015-10-01                   h               1042.48      
                   ...   ...                          ...             ...          
1462073          98857   2015-08-01                   NULL            153.86       
1462074          95776   2015-09-01                   NULL            153.86       
1462075          98857   2015-09-01                   NULL            153.86       
1462076          90001   2015-10-01                   NULL            310.86       
1462077         946957   2015-10-01                   NULL            153.86       


1462078 rows x 4 columns
memory usage: 77.16 MB
name: probehnuto
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/seznam/probehnuto/

In [7]:
probehnuto_mimo_penezenku

name      client_id   Month/Year      probehla_inzerce_mimo_penezenku
  role   unused_float   unused_string   unused_string                  
     0           3901   2012-08-01      ANO                            
     1           3901   2012-09-01      ANO                            
     2           3901   2012-10-01      ANO                            
     3           3901   2012-11-01      ANO                            
     4           3901   2012-12-01      ANO                            
                  ...   ...             ...                            
599381        9804086   2015-10-01      ANO                            
599382        9804238   2015-10-01      ANO                            
599383        9804782   2015-10-01      ANO                            
599384        9804810   2015-10-01      ANO                            
599385        9805032   2015-10-01      ANO                            


599386 rows x 3 columns
memory usage: 23.38 MB
name: probehnuto_mimo_penezenku
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/seznam/probehnuto_mimo_penezenku/

### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
dobito.set_role("client_id", getml.data.roles.join_key)
dobito.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
dobito.set_role("sluzba", getml.data.roles.categorical)
dobito.set_role("kc_dobito", getml.data.roles.numerical)

dobito.set_unit("sluzba", "service")

dobito

name    month_year_datum_transakce   client_id   sluzba        kc_dobito
  role                    time_stamp    join_key   categorical   numerical
  unit   time stamp, comparison only               service                
     0                    2012-10-01     7157857   c               1045.62
     1                    2015-10-01      109700   c               5187.28
     2                    2015-08-01       51508   c                408.2 
     3                    2012-10-01     9573550   c                521.24
     4                    2014-11-01     9774621   c                386.22
                                 ...         ...   ...               ...  
554341                    2012-09-01       65283   g               7850   
554342                    2012-08-01     6091446   g              31400   
554343                    2013-08-01     1264806   g              -8220.52
554344                    2012-08-01      101103   g               3140   
554345                    2012-08-01     8674551   g               6280   


554346 rows x 4 columns
memory usage: 13.30 MB
name: dobito
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/seznam/dobito/

In [9]:
probehnuto.set_role("client_id", getml.data.roles.join_key)
probehnuto.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
probehnuto.set_role("sluzba", getml.data.roles.categorical)
probehnuto.set_role("kc_proklikano", getml.data.roles.target)

probehnuto.set_unit("sluzba", "service")

probehnuto

name    month_year_datum_transakce   client_id   kc_proklikano   sluzba     
   role                    time_stamp    join_key          target   categorical
   unit   time stamp, comparison only                               service    
      0                    2013-06-01      109145          -31.4    c          
      1                    2015-10-01     9804394           37.68   h          
      2                    2015-10-01     9803353          725.34   h          
      3                    2015-10-01     9801753          194.68   h          
      4                    2015-10-01     9800425         1042.48   h          
                                  ...         ...           ...     ...        
1462073                    2015-08-01       98857          153.86   NULL       
1462074                    2015-09-01       95776          153.86   NULL       
1462075                    2015-09-01       98857          153.86   NULL       
1462076                    2015-10-01       90001          310.86   NULL       
1462077                    2015-10-01      946957          153.86   NULL       


1462078 rows x 4 columns
memory usage: 35.09 MB
name: probehnuto
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/seznam/probehnuto/

In [10]:
probehnuto_mimo_penezenku.set_role("client_id", getml.data.roles.join_key)
probehnuto_mimo_penezenku.set_role("Month/Year", getml.data.roles.time_stamp)

probehnuto_mimo_penezenku

name                    Month/Year   client_id   probehla_inzerce_mimo_penezenku
  role                    time_stamp    join_key   unused_string                  
  unit   time stamp, comparison only                                              
     0                    2012-08-01        3901   ANO                            
     1                    2012-09-01        3901   ANO                            
     2                    2012-10-01        3901   ANO                            
     3                    2012-11-01        3901   ANO                            
     4                    2012-12-01        3901   ANO                            
                                 ...         ...   ...                            
599381                    2015-10-01     9804086   ANO                            
599382                    2015-10-01     9804238   ANO                            
599383                    2015-10-01     9804782   ANO                            
599384                    2015-10-01     9804810   ANO                            
599385                    2015-10-01     9805032   ANO                            


599386 rows x 3 columns
memory usage: 14.39 MB
name: probehnuto_mimo_penezenku
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/seznam/probehnuto_mimo_penezenku/

In [11]:
split = getml.data.split.random(train=0.8, test=0.2)
split

0   train
 1   train
 2   train
 3   test 
 4   train
     ...  


infinite number of  rows
type: StringColumnView

## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [12]:
star_schema = getml.data.StarSchema(population=probehnuto, alias="population", split=split)

star_schema.join(
    probehnuto,
    on="client_id",
    time_stamps="month_year_datum_transakce",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

star_schema.join(
    dobito,
    on="client_id",
    time_stamps="month_year_datum_transakce",
)

star_schema.join(
    probehnuto_mimo_penezenku,
    on="client_id", 
    time_stamps=("month_year_datum_transakce",  "Month/Year"),
)

star_schema

data model

  population:
    columns:
    - sluzba: categorical
    - client_id: join_key
    - kc_proklikano: target
    - month_year_datum_transakce: time_stamp

    joins:
    - right: 'probehnuto'
      on: (population.client_id, probehnuto.client_id)
      time_stamps: (population.month_year_datum_transakce, probehnuto.month_year_datum_transakce)
      relationship: 'many-to-many'
      horizon: 86400.0
      lagged_targets: True
    - right: 'dobito'
      on: (population.client_id, dobito.client_id)
      time_stamps: (population.month_year_datum_transakce, dobito.month_year_datum_transakce)
      relationship: 'many-to-many'
      lagged_targets: False
    - right: 'probehnuto_mimo_penezenku'
      on: (population.client_id, probehnuto_mimo_penezenku.client_id)
      time_stamps: (population.month_year_datum_transakce, probehnuto_mimo_penezenku.Month/Year)
      relationship: 'many-to-many'
      lagged_targets: False

  probehnuto:
    columns:
    - sluzba: categorical
    - client_id: join_key
    - kc_proklikano: target
    - month_year_datum_transakce: time_stamp

  dobito:
    columns:
    - sluzba: categorical
    - client_id: join_key
    - kc_dobito: numerical
    - month_year_datum_transakce: time_stamp

  probehnuto_mimo_penezenku:
    columns:
    - client_id: join_key
    - Month/Year: time_stamp
    - probehla_inzerce_mimo_penezenku: unused_string


container

  population
      subset   name            rows   type
  0   test     probehnuto    292833   View
  1   train    probehnuto   1169245   View

  peripheral
      name                           rows   type     
  0   probehnuto                  1462078   DataFrame
  1   dobito                       554346   DataFrame
  2   probehnuto_mimo_penezenku    599386   DataFrame

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

In [13]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,    
    sampling_factor=0.1,
)

feature_selector = getml.predictors.XGBoostRegressor(n_jobs=1, external_memory=True)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [14]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=star_schema.data_model,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    feature_selectors=[feature_selector],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=['XGBoostRegressor'],
         include_categorical=True,
         loss_function=None,
         peripheral=['dobito', 'probehnuto', 'probehnuto_mimo_penezenku'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

### 2.3 Model training

In [15]:
pipe1.check(star_schema.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO__STAGING_TABLE_3 over 'client_id' and 'client_id', there are no corresponding entries for 0.022066% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and DOBITO__STAGING_TABLE_2 over 'client_id' and 'client_id', there are no corresponding entries for 2.250854% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO_MIMO_PENEZENKU__STAGING_TABLE_4 over 'client_id' and 'client_id', there are no corresponding entries for 26.543966% 

In [16]:
pipe1.fit(star_schema.train)

Checking data model...


Staging...
[========================================] 100%


INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO__STAGING_TABLE_3 over 'client_id' and 'client_id', there are no corresponding entries for 0.022066% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and DOBITO__STAGING_TABLE_2 over 'client_id' and 'client_id', there are no corresponding entries for 2.250854% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.
INFO [FOREIGN KEYS NOT FOUND]: When joining POPULATION__STAGING_TABLE_1 and PROBEHNUTO_MIMO_PENEZENKU__STAGING_TABLE_4 over 'client_id' and 'client_id', there are no corresponding entries for 26.543966% of entries in 'client_id' in 'POPULATION__STAGING_TABLE_1'. You might want to double-check your join keys.


Staging...
[======

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=['XGBoostRegressor'],
         include_categorical=True,
         loss_function=None,
         peripheral=['dobito', 'probehnuto', 'probehnuto_mimo_penezenku'],
         predictors=['XGBoostRegressor'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-JIJ8mk'])

url: http://localhost:1709/#/getpipeline/seznam/hyYQRe/0/

### 2.4 Model evaluation

In [17]:
pipe1.score(star_schema.test)



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




date time             set used   target                 mae          rmse   rsquared
0   2022-03-22 21:09:00   train      kc_proklikano    3090.3262    22460.0288     0.8579
1   2022-03-22 21:10:06   test       kc_proklikano    3160.0215    24479.1577     0.7822

### 2.5 featuretools

Unfortunately, featuretools 1.0.0 and 1.1.0 appear to contain a bug, which make it impossible to apply them to this dataset. For this reason, we report the results attained with featuretools 0.9.

### 2.6 Features

The most important feature looks as follows:

In [18]:
pipe1.features.to_sql()[pipe1.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_39";

CREATE TABLE "FEATURE_1_39" AS
SELECT EWMA_1H( t2."kc_proklikano", t1."month_year_datum_transakce" - t2."month_year_datum_transakce, '+1.000000 days'" ) AS "feature_1_39",
       t1.rowid AS rownum
FROM "POPULATION__STAGING_TABLE_1" t1
INNER JOIN "PROBEHNUTO__STAGING_TABLE_3" t2
ON t1."client_id" = t2."client_id"
WHERE t2."month_year_datum_transakce, '+1.000000 days'" <= t1."month_year_datum_transakce"
AND t1."sluzba" = t2."sluzba"
GROUP BY t1.rowid;
```

### 2.7 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.

In [19]:
# Creates a folder named seznam_pipeline containing
# the SQL code.
pipe1.features.to_sql().save("seznam_pipeline")

In [20]:
pipe1.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("seznam_spark")

### 2.8 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | R-squared  | RMSE   | MAE
-------------------- | ---------- | -------| ----
getML: FastProp      |     78.22% | 24,480 | 3,160
featuretools         |     63.24% | 31,655 | 5,167

## 3. Conclusion

We have benchmarked getML against featuretools on a dataset related to online transactions. We have found that getML outperforms featuretools by a wide margin.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.